### Deep Kung-Fu with advantage actor-critic

In this notebook you'll build a deep reinforcement learning agent for Atari [Kung-Fu Master](https://gym.openai.com/envs/KungFuMaster-v0/) that uses a recurrent neural net.

![https://upload.wikimedia.org/wikipedia/en/6/66/Kung_fu_master_mame.png](https://upload.wikimedia.org/wikipedia/en/6/66/Kung_fu_master_mame.png)

In [ ]:
import os, sys

if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/setup_colab.sh -O- | bash
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/week08_pomdp/atari_util.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/week08_pomdp/env_pool.py
    !touch .setup_complete

In [ ]:
# If you are running on a server, launch xvfb to record game videos
# Please make sure you have xvfb installed
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from IPython.display import display

For starters, let's take a look at the game itself:

* Image resized to 42x42 and converted to grayscale to run faster
* Agent sees last 4 frames of game to account for object velocity

In [ ]:
import gym
from atari_util import PreprocessAtari

def make_env():
    env = gym.make("KungFuMasterDeterministic-v0")
    env = PreprocessAtari(
        env, height=42, width=42,
        crop=lambda img: img[60:-30, 15:],
        dim_order='tensorflow',
        color=False, n_frames=4)
    return env

env = make_env()
obs_shape = env.observation_space.shape
n_actions = env.action_space.n

print("Observation shape:", obs_shape)
print("Num actions:", n_actions)
print("Action names:", env.env.env.get_action_meanings())

In [ ]:
s = env.reset()
for _ in range(100):
    s, _, _, _ = env.step(env.action_space.sample())

plt.title('Game image')
plt.imshow(env.render('rgb_array'))
plt.show()

plt.title('Agent observation (4-frame buffer)')
plt.imshow(s.transpose([0, 2, 1]).reshape([42,-1]))
plt.show()

### POMDP setting

The atari game we're working with is actually a POMDP: your agent needs to know timing at which enemies spawn and move, but cannot do so unless it has some memory.

Let's design another agent that has a recurrent neural net memory to solve this. Here's a sketch.

![img](img1_tf.jpg)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, LSTMCell

In [ ]:
class SimpleRecurrentAgent:
    def __init__(self, obs_shape, n_actions):
        """A simple actor-critic agent"""

        # Let's define our computational graph
        # Note: number of units/filters is arbitrary, you can change it at your will

        # Our backbone is denoted by the part before LSTM
        self.backbone = tf.keras.Sequential((
            Conv2D(32, (3, 3), strides=(2, 2), activation='elu'),
            Conv2D(32, (3, 3), strides=(2, 2), activation='elu'),
            Conv2D(32, (3, 3), strides=(2, 2), activation='elu'),
            Flatten(),
            Dense(256, activation='relu'))
        )
        self.lstm_output_size = 256
        self.lstm_cell = LSTMCell(self.lstm_output_size)

        self.logits_head = Dense(n_actions)
        self.state_value_head = Dense(1)

    @property
    def trainable_variables(self):
        return self.backbone.trainable_variables + \
               self.lstm_cell.trainable_variables + \
               self.logits_head.trainable_variables + \
               self.state_value_head.trainable_variables

    def __call__(self, prev_state, obs_t):
        return self.forward(prev_state, obs_t)

    def forward(self, prev_state, obs_t):
        """
        Takes agent's previous hidden state and a new observation,
        returns a new hidden state and whatever the agent needs to learn
        """

        # Apply the whole neural net for one step here.
        # See docs on self.lstm_cell(...).
        # The recurrent cell should take output of the backbone as input.
        <YOUR CODE>

        new_state = <YOUR CODE>
        logits = <YOUR CODE>
        state_value = <YOUR CODE, please sqeeze the last dimention
                       to more convinient shape (n, ) instead of (n, 1)>
        return new_state, (logits, state_value)

    def step(self, prev_state, obs_t):
        """Same as forward except it takes as input and returns numpy arrays
           (or maybe lists of numpy arrays as input, they're allowed as well)"""
        prev_state = [tf.convert_to_tensor(prev_state[0], dtype='float32'),
                      tf.convert_to_tensor(prev_state[1], dtype='float32')]
        obs_t = tf.convert_to_tensor(obs_t, dtype='float32')
        new_state, outputs = self.forward(prev_state, obs_t)
        new_state = (new_state[0].numpy(), new_state[1].numpy())
        outputs = (outputs[0].numpy(), outputs[1].numpy())
        return new_state, outputs

    def get_initial_state(self, batch_size):
        """Return a list of agent memory states at game start.
           Each state is a tf tensor of shape [batch_size, ...]"""
        # feedforward agent has no state
        return [tf.zeros([batch_size, self.lstm_output_size], dtype='float32'),
                tf.zeros([batch_size, self.lstm_output_size], dtype='float32')]

    def sample_actions(self, agent_outputs):
        """pick actions given numeric agent outputs (numpy arrays)"""
        logits, state_values = agent_outputs
        policy = tf.nn.softmax(logits, axis=-1).numpy()
        return [np.random.choice(len(p), p=p) for p in policy]

In [ ]:
n_parallel_games = 5
gamma = 0.99

agent = SimpleRecurrentAgent(obs_shape, n_actions)

In [ ]:
obs = env.reset()[np.newaxis, :].astype('float32')
_, (logits, value) = agent.step(agent.get_initial_state(1), obs)
print("action logits:\n", logits)
print("state values:\n", value)

### Let's play!
Let's build a function that measures agent's average reward.

In [ ]:
def evaluate(agent, env, n_games=1):
    """Plays an a game from start till done, returns per-game rewards """

    game_rewards = []
    for _ in range(n_games):
        # initial observation and memory
        observation = env.reset()
        prev_memories = agent.get_initial_state(1)

        total_reward = 0
        while True:
            observation = observation[np.newaxis, :].astype('float32')
            new_memories, readouts = agent.forward(
                prev_memories, observation)
            action = agent.sample_actions(readouts)

            observation, reward, done, info = env.step(action[0])

            total_reward += reward
            prev_memories = new_memories
            if done:
                break

        game_rewards.append(total_reward)
    return game_rewards

In [ ]:
%%time
import gym.wrappers

with gym.wrappers.Monitor(make_env(), directory="videos", force=True) as env_monitor:
    rewards = evaluate(agent, env_monitor, n_games=3)

print(rewards)

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices

### Training on parallel games

We introduce a class called EnvPool - it's a tool that handles multiple environments for you. Here's how it works:
![img](https://s7.postimg.cc/4y36s2b2z/env_pool.png)

In [ ]:
from env_pool import EnvPool
pool = EnvPool(agent, make_env, n_parallel_games)

We gonna train our agent on a thing called __rollouts:__
![img](img3.jpg)

A rollout is just a sequence of T observations, actions and rewards that agent took consequently.
* First __s0__ is not necessarily initial state for the environment
* Final state is not necessarily terminal
* We sample several parallel rollouts for efficiency

In [ ]:
%%time
# for each of n_parallel_games, take 10 steps
rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(10)

In [ ]:
print("Actions shape:", rollout_actions.shape)
print("Rewards shape:", rollout_rewards.shape)
print("Mask shape:", rollout_mask.shape)
print("Observations shape: ", rollout_obs.shape)

# Actor-critic objective

Here we define a loss function that uses rollout above to train advantage actor-critic agent.


Our loss consists of three components:

* __The policy "loss"__
 $$ \hat J = {1 \over T} \cdot \sum_t { \log \pi(a_t | s_t) } \cdot A_{const}(s,a) $$
  * This function has no meaning in and of itself, but it was built such that
  * $ \nabla \hat J = {1 \over N} \cdot \sum_t { \nabla \log \pi(a_t | s_t) } \cdot A(s,a) \approx \nabla E_{s, a \sim \pi} R(s,a) $
  * Therefore if we __maximize__ J_hat with gradient descent we will maximize expected reward
  
  
* __The value "loss"__
  $$ L_{td} = {1 \over T} \cdot \sum_t { [r + \gamma \cdot V_{const}(s_{t+1}) - V(s_t)] ^ 2 }$$
  * Ye Olde TD_loss from q-learning and alike
  * If we minimize this loss, V(s) will converge to $V_\pi(s) = E_{a \sim \pi(a | s)} R(s,a) $


* __Entropy Regularizer__
  $$ H = - {1 \over T} \sum_t \sum_a {\pi(a|s_t) \cdot \log \pi (a|s_t)}$$
  * If we __maximize__ entropy we discourage agent from predicting zero probability to actions
  prematurely (a.k.a. exploration)
  
  
So we optimize a linear combination of $L_{td}$ $- \hat J$, $-H$
  
```

```

```

```

```

```


__One more thing:__ since we train on T-step rollouts, we can use N-step formula for advantage for free:
  * At the last step, $A(s_t,a_t) = r(s_t, a_t) + \gamma \cdot V(s_{t+1}) - V(s) $
  * One step earlier, $A(s_t,a_t) = r(s_t, a_t) + \gamma \cdot r(s_{t+1}, a_{t+1}) + \gamma ^ 2 \cdot V(s_{t+2}) - V(s) $
  * Et cetera, et cetera. This way agent starts training much faster since it's estimate of A(s,a) depends less on his (imperfect) value function and more on actual rewards. There's also a [nice generalization](https://arxiv.org/abs/1506.02438) of this.


__Note:__ it's also a good idea to scale rollout_len up to learn longer sequences. You may wish set it to >=20 or to start at 10 and then scale up as time passes.

In [ ]:
def select_log_policy_for_actions(log_policy, actions):
    # This code selects the log-probabilities (log pi(a_i|s_i))
    # for those actions that were actually played.
    actions_one_hot = tf.one_hot(actions, n_actions)
    log_policy_for_actions = tf.reduce_sum(log_policy * actions_one_hot, axis=-1)
    return log_policy_for_actions

In [ ]:
def train_on_rollout(states, actions, rewards, is_not_done, prev_memory_states, gamma=0.99):
    """
    Takes a sequence of states, actions and rewards produced by generate_session.
    Updates agent's weights by following the policy gradient above.
    Please use Adam optimizer with default parameters.
    """
    # shape: [batch_size, time, c, h, w]
    states = tf.convert_to_tensor(states, dtype='float32')
    actions = tf.convert_to_tensor(actions, dtype='int32')  # shape: [batch_size, time]
    rewards = tf.convert_to_tensor(rewards, dtype='float32')  # shape: [batch_size, time]
    is_not_done = tf.convert_to_tensor(is_not_done, dtype='float32')  # shape: [batch_size, time]
    rollout_length = rewards.shape[1] - 1

    with tf.GradientTape() as tape:
        # We want to stop gradient here to prevent it from travelling a long distance 
        memory = [tf.stop_gradient(mem_state) for mem_state in prev_memory_states]

        logits = []  # append logit sequence here
        state_values = []  # append state values here
        for t in range(rewards.shape[1]):
            obs_t = states[:, t]

            # use agent to comute logits_t and state values_t.
            # append them to logits and state_values array

            memory, (logits_t, values_t) = <YOUR CODE>

            logits.append(logits_t)
            state_values.append(values_t)

        logits = tf.stack(logits, axis=1)
        state_values = tf.stack(state_values, axis=1)
        policy = tf.nn.softmax(logits, axis=2)
        log_policy = tf.nn.log_softmax(logits, axis=2)

        # select log-probabilities for chosen actions, log pi(a_i|s_i)
        log_policy_for_actions = select_log_policy_for_actions(log_policy, actions)

        # Now let's compute two loss components:
        # 1) Policy gradient objective.
        # Notes: Please don't forget to call stop_gradient on advantage term. Also please use mean, not sum.
        # it's okay to use loops if you want
        J_hat = 0  # policy objective as in the formula for J_hat

        # 2) Temporal difference MSE for state values
        # Notes: Please don't forget to call on V(s') term. Also please use mean, not sum.
        # it's okay to use loops if you want
        value_loss = 0

        cumulative_returns = tf.stop_gradient(state_values[:, -1]) * is_not_done[:, -1]

        for t in reversed(range(rollout_length)):
            r_t = rewards[:, t]                                # current rewards
            # current state values
            V_t = state_values[:, t]
            V_next = tf.stop_gradient(state_values[:, t + 1])  # next state values
            # log-probability of a_t in s_t
            logpi_a_s_t = log_policy_for_actions[:, t]

            # update G_t = r_t + gamma * G_{t+1} as we did in week6 reinforce
            cumulative_returns = G_t = r_t + gamma * cumulative_returns * is_not_done[:, t]

            # Compute temporal difference error (MSE for V(s))
            value_loss += <YOUR CODE>

            # compute advantage A(s_t, a_t) using cumulative returns and V(s_t) as baseline
            advantage = <YOUR CODE>
            advantage = tf.stop_gradient(advantage)

            # compute policy pseudo-loss aka -J_hat.
            J_hat += <YOUR CODE>

        # regularize with entropy
        entropy_reg = <YOUR CODE: compute entropy regularizer, please mind the sign>

        # add-up three loss components and average over time
        loss = -J_hat / rollout_length +\
            value_loss / rollout_length +\
               -0.01 * entropy_reg

        # Gradient descent step
        grads = tape.gradient(loss, agent.trainable_variables)
    <YOUR CODE: apply gradients>

    return loss.numpy()

In [ ]:
optimizer = <YOUR CODE: choose your favorite optimizer>

In [ ]:
# let's test it
memory = list(pool.prev_memory_states)
rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(10)

train_on_rollout(rollout_obs, rollout_actions,
                 rollout_rewards, rollout_mask, memory)

# Train 

just run train step and see if agent learns any better

In [ ]:
from IPython.display import clear_output
from tqdm import trange
from pandas import DataFrame
moving_average = lambda x, **kw: DataFrame(
    {'x': np.asarray(x)}).x.ewm(**kw).mean().values

rewards_history = []

In [ ]:
for i in trange(15000):
    memory = list(pool.prev_memory_states)
    rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(10)
    train_on_rollout(rollout_obs, rollout_actions,
                     rollout_rewards, rollout_mask, memory)

    if i % 100 == 0:
        rewards_history.append(np.mean(evaluate(agent, env, n_games=1)))
        clear_output(True)
        plt.plot(rewards_history, label='rewards', linewidth=3)
        plt.plot(moving_average(np.array(rewards_history),
                                span=10), label='rewards ewma@10', linewidth=3)
        plt.legend(fontsize=13)
        plt.grid()
        plt.show()
        if rewards_history[-1] >= 10000:
            print("Your agent has just passed the minimum homework threshold")
            break

Relax and grab some refreshments while your agent is locked in an infinite loop of violence and death.

__How to interpret plots:__

The session reward is the easy thing: it should in general go up over time, but it's okay if it fluctuates ~~like crazy~~. It's also OK if it reward doesn't increase substantially before some 10k initial steps. However, if reward reaches zero and doesn't seem to get up over 2-3 evaluations, there's something wrong happening.


Since we use a policy-based method, we also keep track of __policy entropy__ - the same one you used as a regularizer. The only important thing about it is that your entropy shouldn't drop too low (`< 0.1`) before your agent gets the yellow belt. Or at least it can drop there, but _it shouldn't stay there for long_.

If it does, the culprit is likely:
* Some bug in entropy computation. Remember that it is $ - \sum p(a_i) \cdot log p(a_i) $
* Your agent architecture converges too fast. Increase entropy coefficient in actor loss. 
* Gradient explosion - just clip gradients and maybe use a smaller network
* Us. Or TensorFlow developers. Or aliens. Or lizardfolk. Contact us on forums before it's too late!

If you're debugging, just run `logits, values = agent.step(batch_states)` and manually look into logits and values. This will reveal the problem 9 times out of 10: you'll likely see some NaNs or insanely large numbers or zeros. Try to catch the moment when this happens for the first time and investigate from there.

### "Final" evaluation

In [ ]:
import gym.wrappers

with gym.wrappers.Monitor(make_env(), directory="videos", force=True) as env_monitor:
    final_rewards = evaluate(agent, env_monitor, n_games=20)

print("Final mean reward", np.mean(final_rewards))

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices